# Mixing BERT with Numerical Features

## 環境設定 建議使用 Colab 跑訓練與預測 並且 執行階段使用 GPU
基本 import 安裝

In [1]:
# !pip install pandas
# !pip install sklearn
# !pip install numpy
# !pip install -U matplotlib
# !pip install transformer
# !pip install multimodal-transformers

## Data Preprocessing

### Read File

In [2]:
import pandas as pd
import numpy as np

# 訓練與測試資料集
train = pd.read_csv('../../raw_data/intern_homework_train_dataset.csv')
test = pd.read_csv('../../raw_data/intern_homework_public_test_dataset.csv')

# 預測資料集
private_test = pd.read_csv('../../raw_data/intern_homework_private_test_dataset.csv')

# 觀看前 5 筆資料
train.head()

,title,created_at,like_count_1h,like_count_2h,like_count_3h,like_count_4h,like_count_5h,like_count_6h,comment_count_1h,comment_count_2h,comment_count_3h,comment_count_4h,comment_count_5h,comment_count_6h,forum_id,author_id,forum_stats,like_count_24h
0,我的排骨湯,2022-10-05 14:20:21 UTC,12,15,15,15,16,18,10,10,10,10,10,10,598518,428921,0.7,26
1,#請益 婚禮穿搭,2022-10-05 14:28:13 UTC,0,0,3,4,4,4,2,5,8,9,9,9,399302,650840,63.9,11
2,無謂的啦啦隊,2022-10-06 07:18:22 UTC,3,7,8,11,12,14,1,1,2,3,3,3,650776,717288,19.2,19
3,文學理論 課本,2022-09-20 11:39:14 UTC,2,7,11,24,26,26,2,2,8,32,38,63,471023,173889,7.9,29
4,一般課程,2022-09-05 10:18:24 UTC,3,7,7,10,10,11,15,26,35,38,48,49,230184,594332,36.2,16


觀察 Feature 種類

In [3]:
train.describe()

,like_count_1h,like_count_2h,like_count_3h,like_count_4h,like_count_5h,like_count_6h,comment_count_1h,comment_count_2h,comment_count_3h,comment_count_4h,comment_count_5h,comment_count_6h,forum_id,author_id,forum_stats,like_count_24h
count,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000
mean,3.097500,5.224340,7.67098,10.249960,12.733960,15.048340,4.463440,6.798680,9.004460,11.088480,12.993100,14.726880,477643.067400,498339.91460,116.125080,45.194840
std,19.274621,24.516945,29.24685,35.367611,43.009398,51.984946,51.510914,60.982037,69.300161,77.946028,87.371928,99.403843,286979.254083,289767.40654,206.109233,180.888108
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,321.000000,2.00000,0.000000,5.000000
25%,0.000000,1.000000,2.00000,2.000000,3.000000,3.000000,0.000000,1.000000,1.000000,1.000000,2.000000,2.000000,174372.000000,243660.75000,16.300000,7.000000
50%,1.000000,3.000000,4.00000,5.000000,5.000000,6.000000,1.000000,3.000000,4.000000,5.000000,5.000000,6.000000,497426.000000,501998.00000,38.000000,13.000000
75%,3.000000,5.000000,7.00000,9.000000,11.000000,12.000000,4.000000,6.000000,9.000000,11.000000,13.000000,14.000000,717594.000000,751721.50000,93.700000,30.000000
max,4044.000000,4797.000000,5098.00000,5362.000000,5822.000000,6588.000000,4902.000000,4923.000000,4928.000000,4928.000000,5969.000000,8031.000000,998778.000000,999998.00000,1128.300000,13297.000000


In [4]:
train.describe(include=np.object)

/tmp/ipykernel_2623/3188520339.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train.describe(include=np.object)


,title,created_at
count,50000,50000
unique,49158,49654
top,.,2022-10-05 05:30:44 UTC
freq,72,8


In [5]:
print('Feature Columns 有',train.columns)

Feature Columns 有 Index(['title', 'created_at', 'like_count_1h', 'like_count_2h',
       'like_count_3h', 'like_count_4h', 'like_count_5h', 'like_count_6h',
       'comment_count_1h', 'comment_count_2h', 'comment_count_3h',
       'comment_count_4h', 'comment_count_5h', 'comment_count_6h', 'forum_id',
       'author_id', 'forum_stats', 'like_count_24h'],
      dtype='object')


In [6]:
text_feature = ['title']
number_feature = ['created_at', 'like_count_1h', 'like_count_2h','like_count_3h', 'like_count_4h', 'like_count_5h', 'like_count_6h',
       'comment_count_1h', 'comment_count_2h', 'comment_count_3h','comment_count_4h', 'comment_count_5h', 'comment_count_6h','forum_stats']
category_feature = ['author_id','forum_id']
print('屬於 文字 的 Feature 有',text_feature)
print('屬於 數字 的 Feature 有',number_feature)
print('屬於 ID / 布林 / 類別 的 Feature 有',category_feature)

屬於 文字 的 Feature 有 ['title']
屬於 數字 的 Feature 有 ['created_at', 'like_count_1h', 'like_count_2h', 'like_count_3h', 'like_count_4h', 'like_count_5h', 'like_count_6h', 'comment_count_1h', 'comment_count_2h', 'comment_count_3h', 'comment_count_4h', 'comment_count_5h', 'comment_count_6h', 'forum_stats']
屬於 ID / 布林 / 類別 的 Feature 有 ['author_id', 'forum_id']


### Feature Selection

挑選 Input

省略掉 作者 ID  因為我認為 BART 沒辦法了解這些 Feature, 會讓模型預測文字輸出帶來 noise.

In [7]:
# 指定要刪除的 column names，並使用 drop 函數將這些 column 刪除

drop_columns = ['author_id']

train_input = train.drop(drop_columns, axis=1)

### Data Transfromations

處理 created_by Feature

In [8]:
# 將文章發佈時間 拆成 星期幾 與 小時 的函數
def split_date(df, date_column):

    # 將 created_by 欄位轉換成日期格式
    df[date_column] = pd.to_datetime(df[date_column], utc=True)
    
    # 新增 星期幾 和 小時 欄位
    df['weekday'] = df[date_column].dt.weekday
    df['hour'] = df[date_column].dt.hour

    # 移除 created_by 欄位
    df = df.drop(date_column, axis=1)

    # 回傳處理過的資料集
    return df

In [9]:
train_input = split_date(train_input, 'created_at')

# 顯示處理過的資料集
train_input.head()

,title,like_count_1h,like_count_2h,like_count_3h,like_count_4h,like_count_5h,like_count_6h,comment_count_1h,comment_count_2h,comment_count_3h,comment_count_4h,comment_count_5h,comment_count_6h,forum_id,forum_stats,like_count_24h,weekday,hour
0,我的排骨湯,12,15,15,15,16,18,10,10,10,10,10,10,598518,0.7,26,2,14
1,#請益 婚禮穿搭,0,0,3,4,4,4,2,5,8,9,9,9,399302,63.9,11,2,14
2,無謂的啦啦隊,3,7,8,11,12,14,1,1,2,3,3,3,650776,19.2,19,3,7
3,文學理論 課本,2,7,11,24,26,26,2,2,8,32,38,63,471023,7.9,29,1,11
4,一般課程,3,7,7,10,10,11,15,26,35,38,48,49,230184,36.2,16,0,10


In [10]:
train_input.columns

Index(['title', 'like_count_1h', 'like_count_2h', 'like_count_3h',
       'like_count_4h', 'like_count_5h', 'like_count_6h', 'comment_count_1h',
       'comment_count_2h', 'comment_count_3h', 'comment_count_4h',
       'comment_count_5h', 'comment_count_6h', 'forum_id', 'forum_stats',
       'like_count_24h', 'weekday', 'hour'],
      dtype='object')

存資料至指定格式(csv檔)

In [11]:
train_df, val_df, test_df = np.split(train_input.sample(frac=1), [int(.8*len(train_input)), int(.9 * len(train_input))])
print('Num examples train-val-test')
print(len(train_df), len(val_df), len(test_df))
train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
test_df.to_csv('test.csv')

Num examples train-val-test
40000 5000 5000


自定義要處理的資料格式

In [12]:
# forum_id forum_stats
text_cols = ['title']
cat_cols = []
numerical_cols = ['weekday', 'hour', 'like_count_1h', 'like_count_2h', 'like_count_3h',
       'like_count_4h', 'like_count_5h', 'like_count_6h', 'comment_count_1h',
       'comment_count_2h', 'comment_count_3h', 'comment_count_4h',
       'comment_count_5h', 'comment_count_6h','forum_stats']

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': 'like_count_24h'
}

Model 參數

In [13]:
from dataclasses import dataclass, field
from typing import Optional

In [14]:
@dataclass
class ModelArguments:
  """
  Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
  """

  model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
  )
  config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
  )
  tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
  )
  cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
  )


@dataclass
class MultimodalDataTrainingArguments:
  """
  Arguments pertaining to how we combine tabular features
  Using `HfArgumentParser` we can turn this class
  into argparse arguments to be able to specify them on
  the command line.
  """

  data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
  column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
  })

  column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
  })

  categorical_encode_type: str = field(default='none',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            'choices': ['ohe', 'binary', 'label', 'none']
                                        })
  numerical_transformer_method: str = field(default='none',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
  task: str = field(default="classification",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

  mlp_division: int = field(default=4,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
  combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
  mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
  numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
  use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
  mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
  gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

  def __post_init__(self):
      assert self.column_info != self.column_info_path
      if self.column_info is None and self.column_info_path:
          with open(self.column_info_path, 'r') as f:
              self.column_info = json.load(f)

In [15]:
import numpy as np
import pandas as pd
import json
import logging
import os
from typing import Optional
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular

2023-04-08 11:04:22.875750: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 11:04:22.978098: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-08 11:04:23.436510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-04-08 11:04:23.436630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or direc

In [16]:
model_args = ModelArguments(
    model_name_or_path='bert-base-chinese'
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    combine_feat_method='concat',
    column_info=column_info_dict,
    task='regression'
)

In [17]:
training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    load_best_model_at_end=True,
    metric_for_best_model="MAPE",
    greater_is_better = False,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    num_train_epochs=3,
    logging_steps=25,
    eval_steps=250,
    eval_accumulation_steps = 1,
)

set_seed(training_args.seed)

In [18]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir,
)

Specified tokenizer:  bert-base-chinese


## Load dataset csv to torch Dataset

In [19]:
from dataclasses import dataclass, field
from typing import Optional

In [20]:
# Get Datasets
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    data_args.data_path,
    data_args.column_info['text_cols'],
    tokenizer,
    categorical_encode_type='none',
    numerical_transformer_method='quantile_normal',
    label_col=data_args.column_info['label_col'],
    # categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token,
)

In [22]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
tabular_config = TabularConfig(num_labels=1,
                            #    cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

In [23]:
model = AutoModelWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertWithTabular: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertWithTabular from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithTabular from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertWithTabular were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['tabular_

In [24]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
)

def regression_metrics(p: EvalPrediction):

    
    pred_labels = np.squeeze(p.predictions[0])
    labels = p.label_ids
    
    mape = mean_absolute_percentage_error(labels,pred_labels)
    mae = mean_absolute_error(labels,pred_labels)
    result = {
        "MAE": mae,
        "MAPE": mape
    }

    return result

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=regression_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
trainer.train()

/user_data/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1875
  Number of trainable parameters = 102270000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: hankystyle. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to ./logs/model_name/checkpoint-625
Configuration saved in ./logs/model_name/checkpoint-625/config.json
Model weights saved in ./logs/model_name/checkpoint-625/pytorch_model.bin
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to ./logs/model_name/checkpoint-1250
Configuration saved in ./logs/model_name/checkpoint-1250/config.json
Model weights saved in ./logs/model_name/checkpoint-1250/pytorch_model.bin
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 

TrainOutput(global_step=1875, training_loss=37095.42175, metrics={'train_runtime': 971.6774, 'train_samples_per_second': 123.498, 'train_steps_per_second': 1.93, 'total_flos': 1.3875128928e+16, 'train_loss': 37095.42175, 'epoch': 3.0})

## Prediction

In [ ]:
import numpy as np
import pandas as pd
import json
import logging
import os
from typing import Optional
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
@dataclass
class ModelArguments:
  """
  Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
  """

  model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
  )
  config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
  )
  tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
  )
  cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
  )


@dataclass
class MultimodalDataTrainingArguments:
  """
  Arguments pertaining to how we combine tabular features
  Using `HfArgumentParser` we can turn this class
  into argparse arguments to be able to specify them on
  the command line.
  """

  data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
  column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
  })

  column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
  })

  categorical_encode_type: str = field(default='none',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            'choices': ['ohe', 'binary', 'label', 'none']
                                        })
  numerical_transformer_method: str = field(default='none',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
  task: str = field(default="classification",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

  mlp_division: int = field(default=4,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
  combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
  mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
  numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
  use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
  mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
  gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

  def __post_init__(self):
      assert self.column_info != self.column_info_path
      if self.column_info is None and self.column_info_path:
          with open(self.column_info_path, 'r') as f:
              self.column_info = json.load(f)

In [ ]:
# forum_id forum_stats
text_cols = ['title']
cat_cols = ['forum_id']
numerical_cols = ['weekday', 'hour', 'like_count_1h', 'like_count_2h', 'like_count_3h',
       'like_count_4h', 'like_count_5h', 'like_count_6h', 'comment_count_1h',
       'comment_count_2h', 'comment_count_3h', 'comment_count_4h',
       'comment_count_5h', 'comment_count_6h','forum_stats']

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': 'like_count_24h'
}

from dataclasses import dataclass, field
from typing import Optional

In [ ]:
model_args = ModelArguments(
    config_name = 'logs/model_name/checkpoint-1875',
    model_name_or_path='bert-base-chinese'
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    combine_feat_method='concat',
    column_info=column_info_dict,
    task='regression'
)

In [ ]:
model = AutoModelWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

In [ ]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir,
)

In [ ]:
# Get Datasets
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    data_args.data_path,
    data_args.column_info['text_cols'],
    tokenizer,
    categorical_encode_type='none',
    numerical_transformer_method='none',
    label_col=data_args.column_info['label_col'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token,
)

In [ ]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
tabular_config = TabularConfig(num_labels=1,
                               cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    load_best_model_at_end=True,
    metric_for_best_model="MAPE",
    greater_is_better = False,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    num_train_epochs=3,
    logging_steps=25,
    eval_steps=250,
    eval_accumulation_steps = 1,
)

set_seed(training_args.seed)



import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
)

def regression_metrics(p: EvalPrediction):

    
    pred_labels = np.squeeze(p.predictions[0])
    labels = p.label_ids
    
    mape = mean_absolute_percentage_error(labels,pred_labels)
    mae = mean_absolute_error(labels,pred_labels)
    result = {
        "MAE": mae,
        "MAPE": mape
    }

    return result

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=regression_metrics,
)

In [ ]:
predictions, labels, metrics = trainer.predict(val_dataset)

In [ ]:
metrics

In [ ]:
pred_labels = np.squeeze(predictions[0])
labels = labels

In [ ]:
pred_labels[:10]

In [ ]:
labels[:10]